In [1]:
# импортируем необходимые библиотеки и модули
import pandas as pd
import numpy as np
import optuna
import random
from functools import partial
from pathlib import Path
from typing import Optional

# импортируем из библиотеки etna нужные классы
from etna.datasets import TSDataset
from etna.loggers import WandbLogger,tslogger
from etna.metrics import MAE, MSE, SMAPE, Sign
from etna.models import CatBoostModelMultiSegment
from etna.pipeline import Pipeline
from etna.transforms import (LagTransform,
                             SegmentEncoderTransform,
                             StandardScalerTransform)

# импортируем модуль предупреждений
import warnings
# включим игнорирование предупреждений
warnings.filterwarnings("ignore")

In [2]:
# функция установки начального значения генератора случайных чисел
def set_seed(seed: int = 42):
    # установка генератора случайных чисел python 
    random.seed(seed)
    # установка генератора случайных чисел numpy
    np.random.seed(seed)

In [3]:
def init_logger(config: dict, project: str = "wandb-sweeps", 
                tags: Optional[list] = ["test", "sweeps"]):
    # создание пустого списка для хранения обьектов WandbLogge
    tslogger.loggers = []
    # создание экземпляра WandbLogge. W&B предоставляет облегченную оболочку для 
    # регистрации экспериментов машинного обучения
    wblogger = WandbLogger(project=project, tags=tags, config=config)
    # добавление обьекта WandbLogge в список
    tslogger.add(wblogger)

In [4]:
def dataloader(file_path: Path, freq: str = "D") -> TSDataset:
    df = pd.read_csv(file_path)
    df = TSDataset.to_dataset(df)
    ts = TSDataset(df=df, freq=freq)
    return ts

In [5]:
# целевая функция Optuna.
def objective(trial: optuna.Trial, 
              metric_name: str, 
              ts: TSDataset, 
              horizon: int,
              lags: int, 
              seed: int):

    # установка начального значения генератора случайных чисел 
    set_seed(seed)

    # создадим экземпляр Pipeline, определив
    # модель и список трансформеров
    pipeline = Pipeline(
        # определим модель
        model=CatBoostModelMultiSegment(
            iterations=trial.suggest_int("iterations", 10, 100),
            depth=trial.suggest_int("depth", 1, 12),
        ),
        # определим список трансформеров
        transforms=[
            StandardScalerTransform("target"),
            SegmentEncoderTransform(),
            LagTransform(in_column="target", 
                         lags=list(range(horizon, horizon + 
                                         trial.suggest_int("lags", 1, lags)))),
        ],
        # установка горизонта для прогнозов
        horizon=horizon,
    )

    # инициализируем WandB logger
    init_logger(pipeline.to_dict())

    # запускаем перекрестную проверку с подсчетом метрик
    metrics, _, _ = pipeline.backtest(ts=ts, metrics=[MAE(), SMAPE(), Sign(), MSE()])
    
    # возвращаем средние значения по найденным метрикам
    return metrics[metric_name].mean()


In [6]:
# функция оптимизации Optuna
def run_optuna(
    # горизонт для прогнозов
    horizon: int = 14,
    # метрика
    metric_name: str = "MAE",
    # URL-адрес базы данных,
    storage: str = "sqlite:///optuna.db",
    # название 
    study_name: Optional[str] = None,
    # количество проходов
    n_trials: int = 5,
    # путь к файлу данных
    file_path: Path = "Data/example_dataset.csv",
    # направление оптимизации
    direction: str = "minimize",
    # частота временного ряда
    freq: str = "D",
    # верхняя граница порядка лага
    lags: int = 24,
    # стартовое значение генератора псевдослучайных чисел
    seed: int = 11,
):

    # загрузка данных
    ts = dataloader(file_path, freq=freq)

    # создание сессии оптимизации Optuna
    study = optuna.create_study(
        # URL базы данных. 
        storage=storage,
        study_name=study_name,
        # объект сэмплера
        sampler=optuna.samplers.TPESampler(
            multivariate=True, group=True),
        # флаг управления поведением
        # при разрешении конфликта
        load_if_exists=True,
        # Направление оптимизации
        direction=direction,
    )

    # запуск оптимизации Optuna
    study.optimize(
        partial(objective, metric_name=metric_name, 
                ts=ts, horizon=horizon, 
                lags=lags, seed=seed), n_trials=n_trials
        )


In [7]:
# Вызов функции run_optuna, запуск оптимизации
run_optuna(file_path='Data/example_dataset.csv')

[I 2022-12-24 21:35:49,031] A new study created in RDB with name: no-name-0570686a-ebad-4772-bffc-84ad194f3f9a
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
wandb: Currently logged in as: gewissta. Use `wandb login --relogin` to force relogin


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.7s remaining:    0.0s


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   58.7s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  2.2min remaining:    0.0s


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.6min finished


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[I 2022-12-24 21:51:05,258] Trial 4 finished with value: 46.57735999084957 and parameters: {'iterations': 34, 'depth': 1, 'lags': 9}. Best is trial 3 with value: 43.29799467213687.


# Больше материалов в рамках подписки вы найдете по адресу https://boosty.to/gewissta